<a href="https://colab.research.google.com/github/abineyan/RAG-with-Llama2/blob/main/RAG_with_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf

In [2]:
!pip install -U langchain-community

In [3]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [4]:
#embeddings
!pip install sentence_transformers

In [5]:
!pip install llama_index

In [6]:
!pip install llama_index

In [7]:
!pip install llama-index-llms-huggingface

In [8]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

In [9]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='0ab94b08-9d4d-4c25-a679-b8f4f06b206e', embedding=None, metadata={'page_label': '1', 'file_name': 'Chapter 01 Updated.pdf', 'file_path': '/content/data/Chapter 01 Updated.pdf', 'file_type': 'application/pdf', 'file_size': 1573892, 'creation_date': '2024-11-12', 'last_modified_date': '2024-11-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Chapter I\nSolve Systems of Linear Equations\n1.1 Introduction\nNumerical methods for solving linear systems of equations can generally be divided into two classes:\n• Direct methods. In the absence of roundo\x00 error such methods would yield the exact solution within a\nﬁnite number of steps.\n• Iterative methods. These are methods that are useful for problems involving special, very large matrices

In [10]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGr

In [12]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
%pip install llama-index-embeddings-langchain

In [19]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

<ipython-input-19-01f3c42df735>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

Settings.llm =llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=1024)

In [26]:
index=VectorStoreIndex.from_documents(documents, embed_model = embed_model, service_context=Settings)

In [27]:
query_engine=index.as_query_engine()

In [28]:
response=query_engine.query("explain fixed point method")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [29]:
print(response)


A fixed point method is a numerical method used to find a fixed point of a function. The method involves iterating the function until the output converges to a single point. This method is useful for finding fixed points of functions that are not necessarily differentiable or continuous.

In the context of the chapter, the fixed point method is used to find the fixed points of a nonlinear system of equations. The method involves defining a function G from the domain of the system to the real numbers, and then iterating the function until a fixed point is found. The fixed point method is a powerful tool for solving nonlinear problems, and it is widely used in many fields, including physics, engineering, and economics.

The fixed point method can be summarized in the following steps:

1. Define a function G from the domain of the system to the real numbers.
2. Iterate the function G until a fixed point is found.
3. The fixed point is the solution to the system of equations.

The fixed p

In [30]:
response=query_engine.query("give an example for fixed point method and solve it")

In [31]:
print(response)

Sure! Let's consider the following nonlinear system of equations:

3x1 ≠ cos(x2x3) ≠ 1
2 =0
x2
1 ≠ 81(x2 +0 .1)2 +s i nx3 +1 .06 = 0,
e≠x1x2 + 20x3 + 10fi ≠ 3
3 =0

We can represent this system of equations in a fixed-point form as x = G(x), where G(x) = (x1, x2, x3)t is a function that satisfies the following conditions:

* G(x) œ D, where D = {(x1, x2, x3)t | x1 ≠ cos(x2x3), x2 ≠ 81(x2 + 0.1)2 + sin(x3) + 1.06, x3 ≠ 3}
* All the component functions of G have continuous partial derivatives and a constant K < 1 exists such that

...gi(x)
...xj
...Œ

